# Introduzione

# Importazione pacchetti

Importare i vari pacchetti necessari  per eseguire le azioni elencate nell'introduzione:
   - selenium (per navigare sulle pagine web)
   - bs4 (beautiful soup, estrarre informazioni dal codice html)
   - pandas (manipolazione/esportazione dataframes)
   - time (per i time sleep ovvero periodi di attesa)
   - tqdm (per le barre di progresso) 

In [ ]:
#%% Pacchetti
## Importazione pacchetti per selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service # Selezionare il chrome driver
#-------##-------##-------##-------##-------##-------##-------##-------##-------##-------#

from bs4 import BeautifulSoup # Per gestire HTML

import os # Muoversi tra directory ecc...
from tqdm import tqdm as tqdm # Barra di progresso


import time   # Per utilizzare dei time.sleep()
import random # Per ottenere numeri casuali, utile per utilizzare time.sleep() randomici (meno identificabili)

# Importare sys per indicare il path della cartella ove sono collocate le funzioni selfmade.
import sys
sys.path.append('.../funzioni')

# Importare le funzioni per lavorare su instagram con il selenium both
from funzioni_selenium_bot_agoda import *

# Percorsi file 
Siccome questo bot leggerà e scriverà file localmente, stabilire quali sono i percorsi file di questi elementi.   

In [ ]:
# Path per il driver di selenium
path_driver="..."

# Directory principale dello scraper, ove vi sono gli script e gli html devono essere posizionati
path_main_directory = '...'

Creare le directory ove mettere i file

In [ ]:
# Andare alla directory principale specificata nella precedente linea di codice
os.chdir(path_main_directory)

# Se non c'è la directory html crearla. Viene inserita questa clausola per evitare che una directory esistente
# venga cancellata
if "html" not in os.listdir():
   os.mkdir("html")

# Impostare il selenium bot e accesso ad Instagram

In [ ]:
# La classe Service di Selenium viene utilizzata per gestire e configurare il driver del browser
service = Service(executable_path=path_driver)

# Configurare il driver di Chrome 
options = webdriver.ChromeOptions()

# Questa impostazionie permette di non caricare le immagini, rende lo scraping più veloce.
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)

# Definire il driver: utilizzare il driver scelto (service) e passargli le impostazioni scelte (options)
driver = webdriver.Chrome(service=service, options=options)

# Impostare le dimensioni della finestra google sulla quale il bot selenium opera.
driver.maximize_window()            # dimensioni massime per la finestra
#driver.set_window_size(600, 800)   # dimensioni customizzate per la finestra

# Istruzioni per il selenium bot 

Il selenium bot per agoda 

In [ ]:
# Specificare il link della pagina dalla quale estrarre i dati.
link = """https://www.agoda.com/search?guid=669ab2ec-48e1-4352-9316-2acd622148c4&asq=u2qcKLxwzRU5NDuxJ0kOF3T91go8JoYYMxAgy8FkBH1BN0lGAtYH25sdXoy34qb9PINtsSLFICq1nQEls9V31kbe5%2FeRO08Xj7Btxoc0FhR9LlVYLumtQbdAjkwbrgpDCpAzJy%2BIPcIhZIlXClDyyvFa0n%2BkEMJafpcscbBJpFTHO8GSTit5qfwwBQvCgLXLW6v%2B1z7ngi%2Fh%2BcIMzh%2Fx3w%3D%3D&city=16594&tick=638855606350&locale=en-us&ckuid=45687004-1d36-4d5b-a134-176448822573&prid=0&currency=EUR&correlationId=21b6ff6b-77a0-4560-836b-1190abb45234&analyticsSessionId=4946736805396046751&pageTypeId=1&realLanguageId=1&languageId=1&origin=IT&stateCode=RM&cid=-1&userId=45687004-1d36-4d5b-a134-176448822573&whitelabelid=1&loginLvl=0&storefrontId=3&currencyId=1&currencyCode=EUR&htmlLanguage=en-us&cultureInfoName=en-us&machineName=am-pc-4h-acm-web-user-69f7876d78-7xhgt&trafficGroupId=4&trafficSubGroupId=4&aid=130243&useFullPageLogin=true&cttp=4&isRealUser=true&mode=production&browserFamily=Safari&cdnDomain=agoda.net&checkIn=2025-08-01&checkOut=2025-08-02&rooms=1&adults=2&children=0&priceCur=EUR&los=1&textToSearch=Rome&travellerType=1&familyMode=off&ds=Nfjv0Uc2AgHSoE4w&productType=-1"""

# Andare alla pagina assegnata alla variabile 'link' 
driver.get(link) 

# Aspettare 10 secondi per far caricare la pagina
time.sleep(10)

# Accettare i cookie
click_cookie_button(driver)


scroll_step = 200
last_height = driver.execute_script("return document.body.scrollHeight")
current_position = 0
scroll = 0


while True:
   # Prendere l'html della pagina 
   html=driver.page_source 
   
   # Trasformare l'html in un oggetto BeautifulSoup. Permette di  estrarre le informazioni dall'html.
   html_bs4 = BeautifulSoup(html,'html.parser')
   

   # Identificare numero di pagine e pagine corrente  
   pagine = html_bs4.find("div",{"id":"paginationContainer"}).find("span",{"id":"paginationPageCount"}).text.replace("Pagina","")
   pagina_corrente = int(pagine.split("di")[0])
   pagine_totali = int(pagine.split("di")[1])

   
   # Salvare l'html in un file txt
   with open(f"{path_main_directory}/html/html_{scroll}.txt","w+") as f:
      f.write(str(html))    
      
    
   driver.execute_script(f"window.scrollBy(0, {scroll_step});")
   
   # Tempo di attesa randomico, meno riconoscibile
   tempo_attesa = random.randint(3,6)
   time.sleep(tempo_attesa)
   
   current_position += scroll_step
   last_height = driver.execute_script("return document.body.scrollHeight")
   scroll += 1
   


   # Se si è arrivati al finale della pagina e si è alla pagina finale interrompere
   if (current_position >= last_height) and (pagina_corrente == pagine_totali) :
      with open(f"{path_main_directory}/html/html_final_pagina{pagina_corrente}.txt","w+") as f:
         f.write(str(html))  
      break 
   

   # se si è arrivati al finale della pagina ma ci sono ancora pagine andare alla pagina successivaa
   elif (current_position >= last_height) and (pagina_corrente < pagine_totali) :
      # Salvare l'html in un file txt, quando si clicca si andrà su una nuova pagina, salvare tutti i risultati
      with open(f"{path_main_directory}/html/html_final_pagina{pagina_corrente}.txt","w+") as f:
         f.write(str(html))  

      current_position = 0
      click_avanti_button(driver)
      time.sleep(20)
      
      print("pagina successiva, si va a pagina:",pagina_corrente+1)
      continue 

      


   
driver.quit()

Cookie accettati metodo 1
Avanti cliccato metodo 1
pagina successiva, si va a pagina: 2
Avanti cliccato metodo 1
pagina successiva, si va a pagina: 3
Avanti cliccato metodo 1
pagina successiva, si va a pagina: 4
Avanti cliccato metodo 1
pagina successiva, si va a pagina: 5
Avanti cliccato metodo 1
pagina successiva, si va a pagina: 6
Avanti cliccato metodo 1
pagina successiva, si va a pagina: 7


KeyboardInterrupt: 